In [14]:
import sys
sys.path.append('../')
import os
os.chdir('..')
import torch
from data import dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from models import distilroberta_classifier as drc
from metrics import eval
from solver import solver_llm
import importlib
import copy
import matplotlib.pyplot as plt
from datetime import datetime
timestamp_str = datetime.now().strftime("%Y%m%d%H%M%S")

In [9]:
train_file="data/train_en.tsv"
test_file="data/dev_en.tsv"
model_name = 'distilroberta_classifer'

tokenizer_dict = {
    'distilroberta_classifer': 'roberta-base'
}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dict[model_name])
device = 'cpu'

train_dataset = dataset.RoBerta_Dataset(train_file,tokenizer,device = device)
test_dataset = dataset.RoBerta_Dataset(test_file,tokenizer,device = device                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 )
train_batch_size = 16
test_batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)


In [89]:
importlib.reload(drc)
importlib.reload(eval)
model = drc.RobertaClassifier(dropout_rate = 0.2 ,device='cpu')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

solver = solver_llm.SolverLLM(model,
                            optimizer,
                            criterion)
max_epoch = 3
## training loop
best = 0.0
best_cm = None
best_model = None
train_acc_epoch = []
valid_acc_epoch = []
for epoch in range(max_epoch):
    solver.epoch = epoch
    # train loop
    train_acc, train_cm = solver_llm.train(train_loader)
    train_acc_epoch.append(train_acc.detach().cpu().numpy())

    # validation loop
    valid_acc, valid_cm = solver_llm.validate_llm(epoch)
    valid_acc_epoch.append(valid_acc.detach().cpu().numpy())

    if valid_acc > best:
        best = valid_acc
        best_cm = valid_cm
        best_model = copy.deepcopy(model)
torch.save(model.save(best_model.state_dict(), 
                '{}_{}'.format(model_name, timestamp_str)))
print('Best Prec @1 Acccuracy: {:.4f}'.format(best))
per_cls_acc = best_cm.diag().detach().numpy().tolist()
for i, acc_i in enumerate(per_cls_acc):
    print("Accuracy of Class {}: {:.4f}".format(i, acc_i))

plt.figure()
plt.plot(range(max_epoch), train_acc_epoch, label='train')
plt.plot(range(max_epoch), valid_acc_epoch, label='validation')
plt.legend()
plt.title("accuracy curve")
plt.xlabel('epoch')
plt.ylabel('accuracy')
# plt.show()

Epoch: [0][0/563]	Time 1.828 (1.828)	Loss 0.7824 (0.7824)	Prec @1 0.3750 (0.3750)	
Train Accuracy of Class 0: 0.3077
Train Accuracy of Class 1: 0.6667
* Train Prec @1: 0.3750
Epoch: [0][10/563]	Time 1.177 (1.357)	Loss 0.6227 (1.1758)	Prec @1 0.7500 (0.5284)	
Train Accuracy of Class 0: 0.5792
Train Accuracy of Class 1: 0.4837
* Train Prec @1: 0.5284


KeyboardInterrupt: 